In [1]:
from model.data import *
from model.model import *
d = Data()

Using TensorFlow backend.


In [ ]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, 64, 64, 64)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

# model.summary()
counter = 1
for layer in model.layers:
    print(counter, "&", layer.name, "&", layer.output_shape)
    counter += 1


In [4]:
# 2 & conv3d_1 & (None, 32, 64, 64, 64)
# 3 & activation_1 & (None, 32, 64, 64, 64)
# 4 & conv3d_2 & (None, 32, 64, 64, 64)
# 5 & activation_2 & (None, 32, 64, 64, 64)
# 6 & max_pooling3d_1 & (None, 32, 32, 32, 32)
counter = 1
filt = 32
lay = 64
while (filt < 512):
#     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    lay = int(lay / 2)
    print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt *= 2
    counter += 1    

    
#     2 & Conv3D + ReLU & (64, 64, 64, 64) \\
#     2 & Max pooling & (64, 32, 32, 32) \\

Contract 1 & Conv3D + ReLU & (32, 64, 64, 64) \\
Contract 1 & Max pooling & (32, 32, 32, 32) \\
Contract 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Contract 2 & Max pooling & (64, 16, 16, 16) \\
Contract 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Contract 3 & Max pooling & (128, 8, 8, 8) \\
Contract 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Contract 4 & Max pooling & (256, 4, 4, 4) \\


In [5]:
lay = 8
counter = 4
filt = 256
while (counter >= 1):
    print("Expand " + str(counter), "&", "Deconvolution3D & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Concatenation & (" + str(filt * 2) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt = int(filt/2)
    lay *= 2
    counter -= 1

Expand 4 & Deconvolution3D & (256, 8, 8, 8) \\
Expand 4 & Concatenation & (512, 8, 8, 8) \\
Expand 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Expand 3 & Deconvolution3D & (128, 16, 16, 16) \\
Expand 3 & Concatenation & (256, 16, 16, 16) \\
Expand 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Expand 2 & Deconvolution3D & (64, 32, 32, 32) \\
Expand 2 & Concatenation & (128, 32, 32, 32) \\
Expand 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Expand 1 & Deconvolution3D & (32, 64, 64, 64) \\
Expand 1 & Concatenation & (64, 64, 64, 64) \\
Expand 1 & Conv3D + ReLU & (32, 64, 64, 64) \\


In [1]:
file = [0,0.6995568632309151,0.300443136745137,0.641574225394111,0.35842577470601467,
0,0.6917949292924795,0.30820507065118585,0.6206784502014253,0.3793215496096634,
0,0.5122266606536341,0.024506303485693017,0.5665436503236269,0.035341769764202315,
1,0.8908735697641708,0.1091264302432897,0.6613966894691689,0.33860331064693017,
1,0.8930363822489554,0.1069636177541553,0.5214700804746457,0.47852991984770205,
1,0.8537605252770137,0.004116246903718444,0.5878799993582673,0.051763444069749955,
2,0.9159979667589924,0.002516496611535728,0.5984852204700285,0.05804231846433461,
0,0.6219784123622163,0.37802158761544574,0.6039238038705703,0.39607619632773355,
0,0.6566300402452215,0.3433699597298631,0.6012501741422588,0.3987498247123139,
3,0.9450022071725936,0.0017314487647162582,0.6129725582430332,0.05504729865490796,
1,0.6385320509304002,0.3614679490629997,0.6039238038705703,0.39607619632773355,
1,0.8843041388688858,0.1156958611322126,0.5047505443990611,0.49524945527279146,
4,0.9633726481762197,0.001211117063420396,0.6265900358262445,0.06264770376838162,
2,0.6385320509303998,0.3614679490629997,0.6039238038705703,0.39607619632773355,
2,0.9239837428195172,0.07601625717960406,0.6145354583426401,0.3854645426668551,
0,0.0029237588018579466,0.99707624129951,0.0009431514442210927,0.9990568491067864,
3,0.6385320509304002,0.3614679490629997,0.6039238038705703,0.39607619632773355,
3,0.9424764050594436,0.057523594939730305,0.5980601013362296,0.40193989890029175,
1,0.00121123332982239,0.998788766886877,0.0009430559095223921,0.9990569444706187,
4,0.6385320509304,0.3614679490629997,0.6039238038705703,0.39607619632773355,
4,0.9543790719582593,0.04562092804156496,0.6020542265350171,0.3979457738912768,
2,0.001211104222059129,0.9987888959605373,0.0009431223059264986,0.9990568780722682,
3,0.0012119931652900045,0.9987880070176389,0.0009440627334086365,0.9990559376434284,
4,0.0012109305222054102,0.9987890696566966,0.0009434468140227362,0.9990565535670893,
]

print(len(file))
for i in range(int(len(file)/5)):
    print(str(file[i * 5] + 1) + ' & ', end="")
    for j in range(1, 5):
        if j != 4:
            print("{0:.4f}".format(file[i*5 + j]) + ' & ', end="")
        else:
            print("{0:.4f}".format(file[i*5 + j]) + ' \\\\')
                

120
1 & 0.6996 & 0.3004 & 0.6416 & 0.3584 \\
1 & 0.6918 & 0.3082 & 0.6207 & 0.3793 \\
1 & 0.5122 & 0.0245 & 0.5665 & 0.0353 \\
2 & 0.8909 & 0.1091 & 0.6614 & 0.3386 \\
2 & 0.8930 & 0.1070 & 0.5215 & 0.4785 \\
2 & 0.8538 & 0.0041 & 0.5879 & 0.0518 \\
3 & 0.9160 & 0.0025 & 0.5985 & 0.0580 \\
1 & 0.6220 & 0.3780 & 0.6039 & 0.3961 \\
1 & 0.6566 & 0.3434 & 0.6013 & 0.3987 \\
4 & 0.9450 & 0.0017 & 0.6130 & 0.0550 \\
2 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
2 & 0.8843 & 0.1157 & 0.5048 & 0.4952 \\
5 & 0.9634 & 0.0012 & 0.6266 & 0.0626 \\
3 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
3 & 0.9240 & 0.0760 & 0.6145 & 0.3855 \\
1 & 0.0029 & 0.9971 & 0.0009 & 0.9991 \\
4 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
4 & 0.9425 & 0.0575 & 0.5981 & 0.4019 \\
2 & 0.0012 & 0.9988 & 0.0009 & 0.9991 \\
5 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
5 & 0.9544 & 0.0456 & 0.6021 & 0.3979 \\
3 & 0.0012 & 0.9988 & 0.0009 & 0.9991 \\
4 & 0.0012 & 0.9988 & 0.0009 & 0.9991 \\
5 & 0.0012 & 0.9988 & 0.0009 & 0.9991 \\


In [ ]:
# 1 & 0.6996 & 0.3004 & 0.6416 & 0.3584 \\
# 1 & 0.6918 & 0.3082 & 0.6207 & 0.3793 \\
# 2 & 0.8909 & 0.1091 & 0.6614 & 0.3386 \\
# 2 & 0.8930 & 0.1070 & 0.5215 & 0.4785 \\


% binary: 
1 & 0.5122 & 0.0245 & 0.5665 & 0.0353 \\
2 & 0.8538 & 0.0041 & 0.5879 & 0.0518 \\
3 & 0.9160 & 0.0025 & 0.5985 & 0.0580 \\
4 & 0.9450 & 0.0017 & 0.6130 & 0.0550 \\
5 & 0.9634 & 0.0012 & 0.6266 & 0.0626 \\

% normal:
1 & 0.6566 & 0.3434 & 0.6013 & 0.3987 \\
2 & 0.8843 & 0.1157 & 0.5048 & 0.4952 \\
3 & 0.9240 & 0.0760 & 0.6145 & 0.3855 \\
4 & 0.9425 & 0.0575 & 0.5981 & 0.4019 \\
5 & 0.9544 & 0.0456 & 0.6021 & 0.3979 \\

% all:
1 & 0.6220 & 0.3780 & 0.6039 & 0.3961 \\
2 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
3 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
4 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
5 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\

